# K Nearest P Median Problem and Implementation

## Introduction

The P-Median Model with Near-Far Cost Allocation, alternatively known as the k nearest p-median Problem, offers a distinctive twist on the traditional p-median problem. This approach enhances our capability to effectively address location allocation challenges. In this notebook, we'll dive into the problem's formulation, and showcase the implementation of the `KNearestPMedian` class in the `spopt` package.

### Understanding the K Nearest P-median Problem

The k nearest p-median problem extends the concept of facility location allocation by considering both nearest and non-nearest facilities. In the article by Church (2018), it proposed this new p-median model which can distinguish between near and far facilities, use both explicit and implicit variables for capacity allocations. By integrating this model, the `spopt` package becomes equipped to deliver heightened precision and efficiency in solving spatial optimization problems, especially those involving extensive datasets and substantial demand volumes.

The model can be formulated as:

$\begin{array}
       \displaystyle \textbf{Minimize} & \displaystyle \sum_{i \in I}\sum_{k \in k_{i}}{a_i d_{ik} X_{ik}} + \sum_{i \in I}{g_i (d_{i{k_i}} + 1)}  &&& (1) \\
       \displaystyle \textbf{Subject To} & \sum_{k \in k_{i}}{X_{ik} + g_i = 1} && \forall i \in I & (2) \\
                                            & \sum_{j \in J}{Y_j} = p                                                                                   &&                                          & (3)                                                                               \\
                                            & \sum_{i \in I}{a_i X_{ik}} \leq {Y_{k} c_{k}}                                                             &&  \forall k \in k_{i}                     & (4)                                                                               \\  
                                            & X_{ij} \leq Y_{j}                                                                                         && \forall i \in I \quad \forall j \in J    & (5)                                                                               \\
                                            & X_{ij} \in \{0, 1\}                                                                                       && \forall i \in I \quad \forall j \in J    & (6)                                                                               \\
                                            & Y_j \in \{0, 1\}                                                                                          && \forall j \in J                          & (7)                                                                               \\
                                            &                                                                                                           &&                                          &                                                                                   \\ \end{array}$
                                            
$\begin{array} \displaystyle \textbf{Where}&& i& =& \textrm{index of demand points/areas/objects in set } I\\
                                            && j                                                                                                        & =                                         & \textrm{index of potential facility sites in set } J                              \\
                                            && p                                                                                                        & =                                         & \textrm{the number of facilities to be sited}                                     \\
                                            && a_i                                                                                                      & =                                         & \textrm{service load or population demand at client location } i                  \\
                                            && k_{i}                                                                                                    & =                                         & \textrm{the } k \textrm{ nearest facilities of client location } i                        \\
                                            && c_{j}                                                                                                    & =                                         & \textrm{the capacity of facility} j                                               \\   
                                            && d_{ij}                                                                                                   & =                                         & \textrm{shortest distance or travel time between locations } i \textrm{ and } j   \\
                                            && X_{ij}                                                                                                   & =                                         & \begin{cases}
                                                                                                                                                                                                       1, \textrm{if client location } i \textrm{ is served by facility } j             \\
                                                                                                                                                                                                       0, \textrm{otherwise}                                                            \\
                                                                                                                                                                                                      \end{cases}                                                                       \\
                                            && Y_j                                                                                                      & =                                         & \begin{cases}
                                                                                                                                                                                                       1, \textrm{if a facility is sited at location } j                                \\
                                                                                                                                                                                                       0, \textrm{otherwise}                                                            \\
                                                                                                                                                                                                      \end{cases}                                                                       \\ 
                                            && g_i                                                                                                      & =                                         & \begin{cases}
                                                                                                                                                                                                       1, \textrm{if the client } i \textrm{ need to be served by non-k-nearest facilities}     \\
                                                                                                                                                                                                       0, \textrm{otherwise}                                                            \\
                                                                                                                                                                                                      \end{cases}                                                                       \\ \end{array}$

This model introduces a clever approach by differentiating between nearby and distant facilities. For instance, at the beginning, only the five closest facilities to each client are considered, which are termed as "near" facilities. If all client demands and constraints can be satisfied, the problem will have an optimal solution right away. However, if the initial facilities can't meet all the demands, we can supplement them with additional nearby facilities.

Mathematically, we introduce a variable called the placeholder facility decision variable, which is $g_i$
in the formula, presenting the distant facilities.

In terms of the solution approach:
1. For each client, identify their five closest facilities.
2. Run the p-median model only for the selected facilities from Step 1 (referred to as "explicitly assigned" in the paper).
3. Check the model outcome. If all $g_i$ values for clients are equal to 0 and an optimal solution exists, we accept this solution.
4. If some $g_i$ values are more than 0, we need to increase the value $k$ for specific clients. Then, rerun the model. Ensure that all $g_i$ values become 0, indicating an optimal solution.

The default starting value for $k$ is 5, or the total number of facilities if it is fewer than 5. Nevertheless, users have the flexibility to customize this value according to their particular requirements.

This model allows us to avoid incorporating the complete set of distances and facilities into the model. Instead, we can focus on integrating smaller segments of these elements and utilize iteration to identify the optimal solution, thereby enhancing computational speed and efficiency.

## The `KNearestPMedian` Class Implementation

We opt to generate a fresh subclass named `KNearestPMedian` within the `spopt` library, derived from the `PMedian` class. This new subclass can inherit all the modules employed by its parent class.

Following this, we will proceed to outline its benefits and features, encompassing three key aspects: 
1. enhanced performance with tree query and sparse matrices,
2. structural composition and logical framework.

### Great Efficiency

The `KNearestPMedian` class exclusively allows the use of **geodataframes** as input data. This approach eliminates the need for users to manually compute distances. A significant enhancement is the utilization of an appropriate **tree data structure**, enabling efficient retrieval of the k nearest facilities for each client. This method significantly reduces computation time by avoiding the calculation and comparison of all distances. Additionally, the resultant distances will be stored in a format of a **sparse matrix**, optimizing memory utilization, particularly for extensive datasets.

### Structure and Logics

The `KNearestPMedian` class and its methods are as following:

| Aspect               | Description                                                                                                                                                                              |
|----------------------|------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **Class Definition** |                                                                                                                                                                                          |
| Class Name           | `KNearestPMedian` (subclass of `PMedian`)                                                                                                                                                |
| Description          | Implements the P-Median Model with Near-Far Cost Allocation and solves it.                                                                                                             |
|                      | Adapted from a specific citation and formulation.                                                                                                                                      |
| **Parameters**       |                                                                                                                                                                                          |
| `weights_sum`        | Sum of weights representing the service loads of clients.                                                                                                                               |
| `clients`            | An array of coordinates of clients.                                                                                                                                                     |
| `facilities`         | An array of coordinates of facilities.                                                                                                                                                  |
| `weights`            | An array of weights representing the service loads of clients.                                                                                                                          |
| `k_array`            | An array of k values representing the number of nearest facilities for each client.                                                                                                    |
| `p_facilities`       | The number of facilities to be located.                                                                                                                                                 |
| `capacities`         | An array of facility capacities (optional).                                                                                                                                             |
| `distance_metric`    | The distance metric used for computing distances between clients and facilities.                                                                                                        |
| `name`               | The problem name.                                                                                                                                                                       |
| **Methods**          |                                                                                                                                                                                          |
| `__init__`           | Constructor method to initialize class attributes and parameters.                                                                                                                      |
| `__add_obj`          | Method to add the objective function to the model.                                                                                                                                     |
| `from_cost_matrix`   | Static method (not supported in this subclass).                                                                                                                                         |
| `_create_sparse_matrix` | Method to create a sparse matrix representing the distances between clients and their k nearest facilities.                                                                           |
| `_update_k_array`    | Method to increase the k value for clients with `g_i` > 0 and update the k array.                                                                                                     |
| `_from_sparse_matrix` | Method to create the problem from the sparse distance matrix.                                                                                                                           |
| `from_geodataframe`  | Class method to create an instance from geodataframes.                                                                                                                                  |
| `facility_client_array` | Method to create a 2D array storing facility to client relationships.                                                                                                                   |
| `solve`              | Method to solve the k nearest p-median model iteratively.                                                                                                                              |
| **Attributes**       |                                                                                                                                                                                          |
| `sparse_matrix`      | Compressed Sparse Row matrix representing the distance matrix between clients and facilities.                                                                                          |
| `aij`                | Compressed Sparse Row matrix storing the weighted cost matrix between clients and facilities.                                                                                          |
| `problem`            | A `pulp` instance of an optimization model with constraints, variables, and an objective function.                                                                                     |
| `fac2cli`            | A 2D array storing facility to client relationships.                                                                                                                                   |
| `cli2fac`            | Inverse of `fac2cli` showing client to facility relationships.                                                                                                                          |

## Use the New Model

### References:

- [Church, R. L. (2018). Tobler’s Law and Spatial Optimization: Why Bakersfield? International Regional Science Review, 41(3), 287–310.](https://doi.org/10.1177/0160017616650612) 